##### 读取 train数据

In [1]:
import pandas as pd

# 读取train数据
processed_data_path = './data/Pro_Train_Data.csv'
df_train = pd.read_csv(processed_data_path , encoding = 'utf-8')
df_train.head()

,ID,Age,Gender,Edu,Query_list
0,22DD920316420BE2DF8D6EE651BA174B,1,1,4,柔和 双沟 女生 中财网 首页 财经 周公 解梦 大全 查询 2345 曹云金 再讽 郭德纲...
1,43CC3AF5A8D6430A3B572337A889AFE4,2,1,3,广州 厨宝 烤箱 世情 人情 雨送 黄昏 花易落 风干 泪痕 厦门 酒店用品 批发市场 不想...
2,E97654BFF5570E2CCD433EA6128EAC19,4,1,0,钻石 之泪 耳机 盘锦 沈阳 旅顺 公交 辽宁 阜新 车牌 baidu k715 k716 ...
3,6931EFC26D229CCFCEA125D3F3C21E57,4,2,3,受欢迎 狗狗 排行榜 场景 范例 三维 绘图 软件 酸奶 一起 壮观 衣服 网站 动漫 绘图...
4,E780470C3BB0D340334BD08CDCC3C71A,2,2,4,干槽症 自愈 太太 万岁 舒心 美国 干槽症 眼皮 下面 怎么回事 麦当劳 旋风 勺子 吉林...


In [2]:
df_train_Age = df_train[['Age' , 'Query_list']]

##### 使用 tf-idf特征模型 

In [3]:
df_train_Age = df_train[['Age' , 'Query_list']]

In [4]:
df_train_Age.head()

,Age,Query_list
0,1,柔和 双沟 女生 中财网 首页 财经 周公 解梦 大全 查询 2345 曹云金 再讽 郭德纲...
1,2,广州 厨宝 烤箱 世情 人情 雨送 黄昏 花易落 风干 泪痕 厦门 酒店用品 批发市场 不想...
2,4,钻石 之泪 耳机 盘锦 沈阳 旅顺 公交 辽宁 阜新 车牌 baidu k715 k716 ...
3,4,受欢迎 狗狗 排行榜 场景 范例 三维 绘图 软件 酸奶 一起 壮观 衣服 网站 动漫 绘图...
4,2,干槽症 自愈 太太 万岁 舒心 美国 干槽症 眼皮 下面 怎么回事 麦当劳 旋风 勺子 吉林...


In [5]:
from sklearn.model_selection import train_test_split

X_Age = df_train_Age['Query_list'].values.tolist()
Y_Age = df_train_Age['Age'].values.tolist()

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 字符级装换为词向量
vec = TfidfVectorizer(
    ngram_range=(1,4) ,
    analyzer='word'   , 

    max_features=20000
)

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import StratifiedKFold
import numpy as np

In [8]:
def stratifiedkfold_cv(x , y , vec_word , clf , shuffle = True , n_folds = 5):
    stratifiedk_fold = StratifiedKFold(n_splits = n_folds , shuffle = shuffle)
    history_score = []
    vec_word.fit(x)
    
    for train_index , test_index in stratifiedk_fold.split(x,y):
        X_train , X_test = x[train_index] , x[test_index]
        y_train , y_test = y[train_index] , y[test_index]
        clf.fit(vec_word.transform(X_train) , y_train)
        history_score.append(clf.score(vec_word.transform(X_test) , y_test))
        print('Please Waiting...')
        
    return history_score

In [ ]:
classifier = MultinomialNB()

history_score = stratifiedkfold_cv( np.array(X_Age) , np.array(Y_Age) , vec , classifier ) 

Please Waiting...
Please Waiting...


In [ ]:

print('max_features = 20000')
print('------------------------')
print( history_score)

#### 以上对Age标签的NB模型处理。使用IF-IDF时，当改变vec的参数时，可以提高性能：
- 当vec是默认参数时：（即analyzer = 'word' ,max_features = 4000 ,时）-->[0.5348, 0.5289, 0.53235, 0.5291, 0.5358]  

- 当vec参数为:      （analyzer = 'word' ,max_features = 20000时） -->[0.5375, 0.53925, 0.5395, 0.54025, 0.53805]  

- 当vec参数为:（analyzer = 'word' ,ngram = (1,2), max_features = 20000时） -->[0.5418, 0.5404, 0.5447, 0.5433, 0.5437] 

 
- 当vec参数为:(ngram_range=(1,4) ,analyzer = 'word' , max_features = 20000时）--> 